# 패키지 다운로드

In [1]:
import pandas as pd
import os
import zipfile

C:\Users\choij\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\choij\anaconda3\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
C:\Users\choij\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 압축파일 해체

In [2]:
base_dir = os.getcwd()
zip_file_path = os.path.join(base_dir, 'Korea_Clinical_Datathon_2024_K-MIMIC.zip')

extracted_dir = os.path.join(base_dir, 'extracted_files')

In [3]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# 해제된 파일 목록 확인
extracted_files = os.listdir(extracted_dir)
print("Extracted files:", extracted_files)

Extracted files: ['EMR']


# 한 개의 폴더만 사용

In [3]:
first_path = os.path.join(extracted_dir, 'EMR/001/433')
csv_list = os.listdir(os.path.join(extracted_dir, 'EMR/001/433'))

In [4]:
print(len(csv_list))   #### 25개의 csv 테이블
print(csv_list[:2])

25
['ADMISSIONS.csv', 'CHARTEVENTS.csv']


# 폴더 안 csv 파일 모두 불러오기

In [5]:
dataframes = {}  ## 커다란 데이터 프레임 안에 모든 csv 파일 할당
for file in csv_list:
    var_name = os.path.splitext(file)[0]
    file_path = os.path.join(first_path, file)
    dataframes[var_name] = pd.read_csv(file_path)

    print(f"{var_name} 데이터프레임이 생성되었습니다.")

ADMISSIONS 데이터프레임이 생성되었습니다.
CHARTEVENTS 데이터프레임이 생성되었습니다.
DATETIMEEVENTS 데이터프레임이 생성되었습니다.
DIAGNOSES_ICD 데이터프레임이 생성되었습니다.


C:\Users\choij\AppData\Local\Temp\ipykernel_4712\2763210986.py:5: DtypeWarning: Columns (2,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes[var_name] = pd.read_csv(file_path)


D_ITEMS 데이터프레임이 생성되었습니다.
D_LABITEMS 데이터프레임이 생성되었습니다.
D_TESTITEMS 데이터프레임이 생성되었습니다.
EDSTAY 데이터프레임이 생성되었습니다.
EMAR 데이터프레임이 생성되었습니다.
EMAR_DETAIL 데이터프레임이 생성되었습니다.
HOSPITAL 데이터프레임이 생성되었습니다.
ICUSTAYS 데이터프레임이 생성되었습니다.
INPUTEVENTS 데이터프레임이 생성되었습니다.
LABEVENTS 데이터프레임이 생성되었습니다.
MEDRECON 데이터프레임이 생성되었습니다.
MICROBIOLOGYEVENTS 데이터프레임이 생성되었습니다.
OUTPUTEVENTS 데이터프레임이 생성되었습니다.
PATIENTS 데이터프레임이 생성되었습니다.
PRESCRIPTIONS 데이터프레임이 생성되었습니다.
PROCEDUREEVENTS 데이터프레임이 생성되었습니다.
PROCEDURES_ICD 데이터프레임이 생성되었습니다.
SERVICES 데이터프레임이 생성되었습니다.
TESTEVENTS 데이터프레임이 생성되었습니다.
TRANSFERS 데이터프레임이 생성되었습니다.
TRIAGE 데이터프레임이 생성되었습니다.


# 데이터 테이블 병합

###  각 테이블에 subject ID 유무 확인

### 모든 파일에 존재하지는 않음 -> 다른 col 활용해 테이블 합칠 것으로 예상

In [6]:
yes = 0
no = 0
for var_name, df in dataframes.items():
    if 'SUBJECT_ID' in df.columns:
        yes += 1
    else:
        no += 1
print(f'subject id col 존재 : {yes} csv 파일')
print(f'subject id col 존재하지 않음 : {no} 개의 csv 파일')  

subject id col 존재 : 19 csv 파일
subject id col 존재하지 않음 : 6 개의 csv 파일


### Admission table 에 Hadm ID, Submission ID 고유 개수 다름
### Submission ID: 환자에 대한 고유 ID, Hadm ID: 환자의 방문 고유 ID

In [7]:
print(dataframes['ADMISSIONS']['SUBJECT_ID'])  ## Admission 내 총 590개의 row, 586 명의 환자 존재
print(len(dataframes['ADMISSIONS']['SUBJECT_ID']))  ### 5번 정도가 중복 방문 
print(len(dataframes['ADMISSIONS']['SUBJECT_ID'].unique()))

print('-' * 30)
print(len(dataframes['ADMISSIONS']['HADM_ID']))  ### 5번 정도가 중복 방문 
print(len(dataframes['ADMISSIONS']['HADM_ID'].unique()))

0      10900087449
1      10900007761
2      10900013857
3      10900044937
4      10900014506
          ...     
585    10900072333
586    10900076360
587    10900093353
588    10900093700
589    10900063324
Name: SUBJECT_ID, Length: 590, dtype: int64
590
586
------------------------------
590
590


## 병합에 사용할 Col

### 각 테이블에서 Hadm ID, Chart time, Item ID, Value 값 저장
### 만약 한 col이라도 없을 시 병합x
### Item ID: 어떤 measurement (e.g., heart rate) 에 대한 encoding
### Chart time: 환자 기록 측정 시간
### Value: Item ID 에 대해 측정된 값

#### 이 외의 성별, 입장시간은 후에 병합 예정

## ChartEvent table

### ChartEvent 는 ICU 환자에서 추출한 모든 정보 포함

In [108]:
print(dataframes['CHARTEVENTS']['ITEMID'].nunique())
dataframes['CHARTEVENTS']['ITEMID']

740


0          001C_1775_21185
1          001C_1775_21185
2          001C_1961_20350
3          001C_1961_26610
4          001C_1961_20370
                ...       
4265153    001C_2009_24545
4265154    001C_1818_26965
4265155    001C_1693_26980
4265156    001C_1605_26950
4265157    001C_1395_24765
Name: ITEMID, Length: 4265158, dtype: object

In [8]:
print(dataframes['CHARTEVENTS'].columns)
print(len(dataframes['CHARTEVENTS'])) 
print(len(dataframes['CHARTEVENTS']['HADM_ID'].unique()))
## ChartEvent 는 환자가 받은 모든 측정치가 들어있음 -> 시간 전처리 위해 Admission col 추가할 필요 있음   
## 논문에서 Admission 30 분 까지의 데이터는 사용 x

Index(['CHARTEVENT_ID', 'SUBJECT_ID', 'HADM_ID', 'STAY_ID', 'CHARTTIME',
       'STORETIME', 'ITEMID', 'VALUE', 'VALUENUM', 'VALUEUOM', 'WARNING'],
      dtype='object')
4265158
590


### Q:Value  에 int 값이 아니라 str 값이 많은데 이를 다 int 혹은 binary 로 변환 필요?

In [9]:
my_df = pd.DataFrame()  ## 테이블 병합하기 위한 새로운 테이블 만들기
my_df = dataframes['CHARTEVENTS']  # 가장 많은 chartevent 테이블을 복사
my_df

,CHARTEVENT_ID,SUBJECT_ID,HADM_ID,STAY_ID,CHARTTIME,STORETIME,ITEMID,VALUE,VALUENUM,VALUEUOM,WARNING
0,11001242480370,10900038639,001109000386394405A09C6048648A,164328.0,2411-05-19T00:00:00,2411-05-18T23:11:41,001C_1775_21185,양호함,0.0,NaN,0
1,11001242586738,10900038639,001109000386394405A09C6048648A,164328.0,2411-05-19T08:00:00,2411-05-19T07:18:47,001C_1775_21185,양호함,0.0,NaN,0
2,11001242480415,10900038639,001109000386394405A09C6048648A,164328.0,2411-05-19T00:00:00,2411-05-18T23:11:41,001C_1961_20350,음성,0.0,NaN,0
3,11001242480416,10900038639,001109000386394405A09C6048648A,164328.0,2411-05-19T00:00:00,2411-05-18T23:11:41,001C_1961_26610,음성,0.0,NaN,0
4,11001242480414,10900038639,001109000386394405A09C6048648A,164328.0,2411-05-19T00:00:00,2411-05-18T23:11:41,001C_1961_20370,평가가능,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
4265153,11001276267461,10900019700,001109000197006ADD0C2633B5765F,165674.0,2664-05-10T16:00:00,2664-05-10T15:22:19,001C_2009_24545,"ANUS ***, T-CAN SITE ***",0.0,NaN,0
4265154,11001265292205,10900059291,001109000592911A2EC3C2CC17EC8A,165335.0,2309-07-17T19:00:00,2309-07-17T19:19:09,001C_1818_26965,OFF D/T OR ***,0.0,cc/hr,0
4265155,11001265292206,10900059291,001109000592911A2EC3C2CC17EC8A,165335.0,2309-07-17T19:00:00,2309-07-17T19:19:09,001C_1693_26980,OFF D/T OR ***,0.0,cc/hr,0
4265156,11001265292207,10900059291,001109000592911A2EC3C2CC17EC8A,165335.0,2309-07-17T19:00:00,2309-07-17T19:19:09,001C_1605_26950,OFF D/T OR ***,0.0,cc/hr,0


### 필요 없는 col 삭제

### 가시성 위해 Hadm ID 로 오름차순

In [110]:
my_df = dataframes['CHARTEVENTS'][['HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUE']]

my_df = my_df.sort_values(by = 'HADM_ID', ascending = True) 
my_df

,HADM_ID,CHARTTIME,ITEMID,VALUE
4172744,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
4171315,00110900002214B06A868896938A2A,2904-04-08T16:00:00,001C_1830_21930,Rt. jugular
4171314,00110900002214B06A868896938A2A,2904-04-08T12:00:00,001C_1830_21930,Rt. jugular
4171313,00110900002214B06A868896938A2A,2904-04-08T11:05:00,001C_1830_21930,Rt. jugular
4171312,00110900002214B06A868896938A2A,2904-04-08T07:23:00,001C_2096_26075,Weak
...,...,...,...,...
1117164,00110900094340A1C6B97BCFCFF77B,2752-05-04T20:00:00,001C_1826_27315,easy
1117165,00110900094340A1C6B97BCFCFF77B,2752-05-07T00:00:00,001C_1026_26520,36.6
1117166,00110900094340A1C6B97BCFCFF77B,2752-05-07T05:00:00,001C_1026_26520,37.0
1117114,00110900094340A1C6B97BCFCFF77B,2752-05-05T04:00:00,001C_1676_24755,abd


### 후에 추가로 전처리 할 것이기 때문에 Value 제외 모두 제거
### 필요한 col 인 Value uom 은 일단 저장해두고 나중에 사용

## Admission table
### Admission 테이블에서 Admission 시간 가져와서 각 환자 방문에 대해 입장 시간 저장
### 현재는 병합x, 그러나 나중에 필요할 듯

In [14]:
ADM = dataframes['ADMISSIONS'][['HADM_ID', 'ADMITTIME']]
print(len(ADM))
ADM.head(10) ## 각각의 병원 방문에 대해 admit 타임 기록

590


,HADM_ID,ADMITTIME
0,00110900087449907E04D4E63015F8,2371-07-01T17:58:12
1,001109000077615403EFFAE95253FA,2183-09-02T15:47:53
2,001109000138570B8DCC1558C0846B,2148-09-12T14:59:36
3,00110900044937B1673AC1FC3379AB,2254-08-19T12:49:01
4,0011090001450633CB98B097F2508D,2413-07-12T13:49:02
5,00110900087356FCF3549864C9863F,2368-07-20T12:48:56
6,00110900081317B36427441B857EB6,2751-04-20T14:32:19
7,00110900074905AE14D056725CEAB8,2801-04-04T18:44:06
8,001109000613356DCE7C1859B5679D,2829-03-08T08:26:36
9,001109000204261546203818BC4A1E,2628-05-22T12:08:52


## D _ Items table

### Item ID 에 대한 정보를 담고 있는 D_items table
### Item ID 에 대해 범위 담고 있으므로 필요하겠지만 현재는 병합 X

In [111]:
print(dataframes['D_ITEMS']['ITEMID'].nunique()) #환자
dataframes['D_ITEMS'].head()

65850


,ITEMID,LABEL,ABBREVIATION,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,LOWNORMALVALUE,HIGHNORMALVALUE
0,001C_1001_20640,EKG > EKG 분류,NaN,chartevents,NaN,NaN,Coded Text,0,0
1,001C_1001_22445,EKG > 각차단 종류,NaN,chartevents,NaN,NaN,Coded Text,0,0
2,001C_1001_23435,EKG > 기타 부정맥 종류,NaN,chartevents,NaN,NaN,Coded Text,0,0
3,001C_1001_23705,EKG > 동성 부정맥 종류,NaN,chartevents,NaN,NaN,Coded Text,0,0
4,001C_1001_23900,EKG > 방실 접합부 부정맥 종류,NaN,chartevents,NaN,NaN,Coded Text,0,0


In [16]:
x1 = dataframes['D_ITEMS']['ITEMID']
len(my_df[my_df['ITEMID'].isin(x1)])  ## 모든 Item ID 에 대한 정보가 D _Items 테이블에 존재함을 확인

4265158

In [17]:
ITEMINFO = dataframes['D_ITEMS'][['ITEMID', 'LABEL', 'UNITNAME', 'LOWNORMALVALUE', 'HIGHNORMALVALUE']]
print(len(ITEMINFO))
ITEMINFO.head(10)

65850


,ITEMID,LABEL,UNITNAME,LOWNORMALVALUE,HIGHNORMALVALUE
0,001C_1001_20640,EKG > EKG 분류,NaN,0,0
1,001C_1001_22445,EKG > 각차단 종류,NaN,0,0
2,001C_1001_23435,EKG > 기타 부정맥 종류,NaN,0,0
3,001C_1001_23705,EKG > 동성 부정맥 종류,NaN,0,0
4,001C_1001_23900,EKG > 방실 접합부 부정맥 종류,NaN,0,0
5,001C_1001_25110,EKG > 심방성 부정맥 종류,NaN,0,0
6,001C_1001_25120,EKG > 심실상성 부정맥 종류,NaN,0,0
7,001C_1001_25125,EKG > 심실성 부정맥 종류,NaN,0,0
8,001C_1001_25840,EKG > 자극전도장애 종류,NaN,0,0
9,001C_1002_27090,mean BP(계산) > 평균혈압(계산),mmHg,0,0


## Date Time Events table
### ICU 환자의 측정치를 담고 있는 테이블
### Date Time Events  테이블에서 확인한 결과 D_items 테이블에서의 Item ID와 겹치는 부분 없음
#### (아마 테이블마다 고유의 Item ID 가지고 있는 것으로 예상)
### 기존 테이블에 행으로 넣기

In [18]:
dataframes['DATETIMEEVENTS'].head()

,DATETIMEEVENT_ID,SUBJECT_ID,HADM_ID,STAY_ID,CHARTTIME,STORETIME,ITEMID,VALUE,VALUEUOM,WARNING
0,001NR1000274878077.0,10900079617,00110900079617D8F1412FCFEF91F1,NaN,2155-10-07T08:46:00,2155-10-07T09:26:44,001D_9379_12845,priming함,NaN,0
1,001NR1000274690913.0,10900075399,00110900075399955DB5D76CA0870B,NaN,2448-07-26T05:00:00,2448-07-26T05:27:33,001D_37_11375,간간이 수면 취하고 있음,NaN,0
2,001NR1000274492253.0,10900047121,00110900047121B3C33F9602BC5885,NaN,2617-06-14T21:25:00,2617-06-14T21:40:24,001D_6095_12010,의사에게 알림,NaN,0
3,001NR1000274629745.0,10900017686,001109000176860C5B51DE83C4F9A5,166710.0,2339-08-22T16:59:00,2339-08-22T17:20:37,001D_3351_10685,빛반사 확인결과 prompt함,NaN,0
4,001NR1000272108522.0,10900017686,001109000176860C5B51DE83C4F9A5,NaN,2339-08-04T09:18:00,2339-08-04T09:30:08,001D_3994_10685,동공크기 :,NaN,0


In [19]:
x1 = dataframes['DATETIMEEVENTS']['ITEMID']

common_subjects = my_df[my_df['ITEMID'].isin(x1)]
common_subjects  ## lab event 에서 겹치는 코드는 x

,HADM_ID,CHARTTIME,ITEMID,VALUE


In [20]:
items = dataframes['DATETIMEEVENTS'][['HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUE']]

my_df_2 = pd.concat([my_df, items], ignore_index=True) 
my_df_2

,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T16:00:00,001C_1830_21930,Rt. jugular
2,00110900002214B06A868896938A2A,2904-04-08T12:00:00,001C_1830_21930,Rt. jugular
3,00110900002214B06A868896938A2A,2904-04-08T11:05:00,001C_1830_21930,Rt. jugular
4,00110900002214B06A868896938A2A,2904-04-08T07:23:00,001C_2096_26075,Weak
...,...,...,...,...
5845073,00110900088503E727C4F3BC96231F,2955-09-06T13:53:00,001D_8200_22385,SIMV(PC)+PSV
5845074,001109000575903647F190B6E8CD34,2488-03-13T00:43:00,001D_3994_23675,round
5845075,00110900027557126B9DA290D47DA1,2711-05-08T23:20:00,001D_9567_22365,16
5845076,00110900088503E727C4F3BC96231F,2955-09-13T01:36:00,001D_7355_24505,7.5


In [21]:
dataframes['DATETIMEEVENTS']

,DATETIMEEVENT_ID,SUBJECT_ID,HADM_ID,STAY_ID,CHARTTIME,STORETIME,ITEMID,VALUE,VALUEUOM,WARNING
0,001NR1000274878077.0,10900079617,00110900079617D8F1412FCFEF91F1,NaN,2155-10-07T08:46:00,2155-10-07T09:26:44,001D_9379_12845,priming함,NaN,0
1,001NR1000274690913.0,10900075399,00110900075399955DB5D76CA0870B,NaN,2448-07-26T05:00:00,2448-07-26T05:27:33,001D_37_11375,간간이 수면 취하고 있음,NaN,0
2,001NR1000274492253.0,10900047121,00110900047121B3C33F9602BC5885,NaN,2617-06-14T21:25:00,2617-06-14T21:40:24,001D_6095_12010,의사에게 알림,NaN,0
3,001NR1000274629745.0,10900017686,001109000176860C5B51DE83C4F9A5,166710.0,2339-08-22T16:59:00,2339-08-22T17:20:37,001D_3351_10685,빛반사 확인결과 prompt함,NaN,0
4,001NR1000272108522.0,10900017686,001109000176860C5B51DE83C4F9A5,NaN,2339-08-04T09:18:00,2339-08-04T09:30:08,001D_3994_10685,동공크기 :,NaN,0
...,...,...,...,...,...,...,...,...,...,...
1579915,001ND1000246731835.022385,10900088503,00110900088503E727C4F3BC96231F,160032.0,2955-09-06T13:53:00,2955-09-06T14:02:20,001D_8200_22385,SIMV(PC)+PSV,NaN,0
1579916,001ND1000257040057.023675,10900057590,001109000575903647F190B6E8CD34,162955.0,2488-03-13T00:43:00,2488-03-13T00:43:53,001D_3994_23675,round,NaN,0
1579917,001ND1000272427064.022365,10900027557,00110900027557126B9DA290D47DA1,165492.0,2711-05-08T23:20:00,2711-05-08T23:21:03,001D_9567_22365,16,회/min,0
1579918,001ND1000247631104.024505,10900088503,00110900088503E727C4F3BC96231F,160032.0,2955-09-13T01:36:00,2955-09-13T01:38:55,001D_7355_24505,7.5,cm,0


In [22]:
DATETIME_ITEMINFO = dataframes['DATETIMEEVENTS'][['ITEMID', 'VALUEUOM']]
print(DATETIME_ITEMINFO['ITEMID'].nunique())
print(DATETIME_ITEMINFO['VALUEUOM'].nunique())
DATETIME_ITEMINFO.head(5)

5641
41


,ITEMID,VALUEUOM
0,001D_9379_12845,NaN
1,001D_37_11375,NaN
2,001D_6095_12010,NaN
3,001D_3351_10685,NaN
4,001D_3994_10685,NaN


### 병합한 이후 Hadm ID 로 다시 정렬 및 섞여진 index 초기화

In [117]:
my_df_3 = my_df_2.sort_values(by = 'HADM_ID', ascending = True)
my_df_3.reset_index(drop =True, inplace = True)
my_df_3

,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08T17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08T19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08T18:15:00,001C_1729_22095,13.0
...,...,...,...,...
5845073,00110900094340A1C6B97BCFCFF77B,2752-05-05T08:00:00,001C_1675_27355,crackle
5845074,00110900094340A1C6B97BCFCFF77B,2752-05-05T00:00:00,001C_1675_27120,both lung
5845075,00110900094340A1C6B97BCFCFF77B,2752-05-04T16:00:00,001C_1675_27355,crackle
5845076,00110900094340A1C6B97BCFCFF77B,2752-05-05T08:00:00,001C_1012_24795,105


## D _ Labitems table
### 모든 Lab measurement 관련 데이터 담고 있는 테이블
### D_labitems table에서 겹치는 item ID  있는지 확인
### -> D-labitem 에는 시간 정보도 없기 때문에 병합x

In [24]:
x1 = dataframes['D_LABITEMS']
print(len(x1))
x1.head()

4183


,ITEMID,LABEL,FLUID,CATEGORY,EDI_CODE
0,001L2232,Factor assay (VII),Citrate BLD(B),진단검사의학과및 기타 검체검사 > 혈액응고,D113103D
1,001L2250,(검사중단)Protein S,Citrate BLD,진단검사의학과및 기타 검체검사 > 혈액응고,D1151002
2,001L2694,"(검사중단)MPD panel, FISH",Heparin BM,진단검사의학과및 기타 검체검사 > FISH검사,"C584104B,C584104D,C584112B,C584112D,C584114B,C..."
3,001L4065,"Infection control test, dental unit water culture",Dental water,진단검사의학과및 기타 검체검사 > 일반미생물,KMM90000
4,001L2619,"Williams, FISH",Heparin PB,진단검사의학과및 기타 검체검사 > FISH검사,C5841


In [25]:
x1 = dataframes['D_LABITEMS']['ITEMID']
common_subjects = my_df_3[my_df_3['ITEMID'].isin(x1)]
common_subjects

,HADM_ID,CHARTTIME,ITEMID,VALUE


## D _ Testitem table
### D_tesitem table 에서 겹치는 ITem ID 있는지 확인
### -> 시간 정보 없기 때문에 병합 x

In [26]:
x1 = dataframes['D_TESTITEMS']
print(len(x1))
x1.head()

7934


,ITEMID,LABEL,CATEGORY,EDI_CODE
0,001RC3042,"Chest HRCT (local, nodule densitometry)",영상의학과 > CT > Chest,KMM90000
1,001RG8061,Read Outside Film (단순촬영3매이내),영상의학과 > 일반촬영 > Others,KMM90000
2,001RG8062,Read Outside Film (단순촬영4-10매),영상의학과 > 일반촬영 > Others,KMM90000
3,001RG8063,Read Outside Film (단순촬영11매이상),영상의학과 > 일반촬영 > Others,KMM90000
4,001RG807,Portable (일반),영상의학과 > 일반촬영 > Others,KMM90000


In [27]:
x1 = dataframes['D_TESTITEMS']['ITEMID']
common_subjects = my_df_3[my_df_3['ITEMID'].isin(x1)]
common_subjects  ## 측정치에 대한 고유 ID 가 하나도 겹치지 않기 때문에 Test items table 에서는 결합 x

,HADM_ID,CHARTTIME,ITEMID,VALUE


## Diagnoses ICD table
### Diagnoses ICD 테이블에서 겹치는 Item ID 있는지 확인
### 확인한 결과 없고 시간 정보도 없기 때문에 병합x

In [118]:
x1 = dataframes['DIAGNOSES_ICD']
print(len(x1))
x1.head()

2361


,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD_CODE,ICD_VERSION,ITEMID,IS_ICU
0,10900087966,0011090008796621402C539C492BC3,1,C419,KCD8,001D00001759,Y
1,10900087966,0011090008796621402C539C492BC3,2,C402,KCD8,001D00001730,Y
2,10900087966,0011090008796621402C539C492BC3,3,Z316,KCD8,001D00021290,Y
3,10900087966,0011090008796621402C539C492BC3,4,C780,KCD8,001D00002974,Y
4,10900087966,0011090008796621402C539C492BC3,5,I7431,KCD8,001D00010315,Y


In [28]:
x1 = dataframes['DIAGNOSES_ICD']['ITEMID']
common_subjects = my_df_3[my_df_3['ITEMID'].isin(x1)]
common_subjects

,HADM_ID,CHARTTIME,ITEMID,VALUE


## ED Stay table
### ED Stay table은 환자의 응급실의 입장 시간, 나타난 질병을 나타냄
### 고유 환자인 590명 중 절반도 안되는 환자의 성별밖에 모르므로 활용 x
### Q: 응급실, ICU 는 다른 것?

In [29]:
print(len(dataframes['EDSTAY']))
dataframes['EDSTAY'].head()

184


,SUBJECT_ID,STAY_ID,INTIME,OUTTIME,SEX,DX1,DX1_ICD,DX2,DX2_ICD,DX3,...,DX5,DX5_ICD,DX6,DX6_ICD,DX7,DX7_ICD,DX8,DX8_ICD,DX9,DX9_ICD
0,10900061335,001109000613356DCE7C1859B5679D,2829-03-08T00:50:00,2829-03-08T14:01:23,M,"Coronavirus disease (COVID-19), virus identified",U071,Bronchiolitis obliterans severe,J4482,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10900020426,001109000204261546203818BC4A1E,2628-05-21T20:45:00,2628-05-22T12:54:21,F,"Subdural hematoma, traumatic",S065,Dizziness,R42,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10900084933,001109000849333BAEA5DFC1C9C530,2560-05-22T19:54:00,2560-05-23T21:27:11,F,Stroke,I64,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10900011804,0011090001180456B85EBD007F3149,2879-03-17T09:14:00,2879-03-17T18:36:06,M,Dyspnea,R060,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10900086019,00110900086019BDD664499C01200B,2117-09-20T10:02:00,2117-09-20T16:26:56,F,"Cerebellar neoplasm, uncertain behavior",D431,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
x1 = dataframes['EDSTAY'][['SUBJECT_ID', 'SEX']]
print(len(x1))  ## 총 183 명의 응급실 환자
x1 = x1.drop_duplicates(['SUBJECT_ID'])
x1.head()

184


,SUBJECT_ID,SEX
0,10900061335,M
1,10900020426,F
2,10900084933,F
3,10900011804,M
4,10900086019,F


## EMAR table
### EMAR 테이블은 약 처방에 대한 정보 담고 있음
### Chart time 도 있으니 담을 필요 있음
#### 해당 테이블에는 value col 없는데 우선 medication col 을 value 로 바꿔 병합

In [31]:
print(len(dataframes['EMAR']))
dataframes['EMAR'].head()  ## 약 처방에 대한 정보, CHART TIME도 있으니 저장필요

426543


,SUBJECT_ID,HADM_ID,EMAR_ID,EMAR_SEQ,POE_ID,PHARMACY_ID,CHARTTIME,MEDICATION,EVENT_TXT,STORETIME,EMAR_TYPE,ITEMID,STAY_ID
0,10900028697,001109000286972AC82E9782F34C3A,001A0000000000000000085349479,1,NaN,0010000000000000000421752585,2421-05-15T00:00:00,Sentil 5mg tab(Clobazam),Not given,2421-05-16T19:54:35,Acting,001200501890065450189,NaN
1,10900028697,001109000286972AC82E9782F34C3A,001A0000000000000000085266083,2,NaN,0010000000000000000421664584,2421-05-15T01:00:00,Tazoperan 4.5g inj(Piperacillin/Tazobactam),Not given,2421-05-15T06:30:56,Acting,001880643303671350162,NaN
2,10900028697,001109000286972AC82E9782F34C3A,001A0000000000000000085266086,3,NaN,0010000000000000000421664585,2421-05-15T01:00:00,Normal saline 50mL bag 이노엔,Not given,2421-05-15T06:31:04,Acting,001880100768069950130,NaN
3,10900028697,001109000286972AC82E9782F34C3A,001A0000000000000000085279294,4,NaN,0010000000000000000421665385,2421-05-15T01:01:00,Tazoperan 4.5g inj(Piperacillin/Tazobactam),Not given,2421-05-15T11:13:40,Acting,001880643303671350162,NaN
4,10900028697,001109000286972AC82E9782F34C3A,001A0000000000000000085279295,5,NaN,0010000000000000000421665386,2421-05-15T01:01:00,Normal saline 50mL bag 이노엔,Not given,2421-05-15T11:13:40,Acting,001880100768069950130,NaN


In [33]:
x1 = dataframes['EMAR']['ITEMID']
my_df_3[my_df_3['ITEMID'].isin(x1)]

,HADM_ID,CHARTTIME,ITEMID,VALUE


In [35]:
cols = my_df_3.columns & dataframes['EMAR'].columns
x1 = dataframes['EMAR'][cols]
x1.head()

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\138827579.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = my_df_3.columns & dataframes['EMAR'].columns


,HADM_ID,CHARTTIME,ITEMID
0,001109000286972AC82E9782F34C3A,2421-05-15T00:00:00,001200501890065450189
1,001109000286972AC82E9782F34C3A,2421-05-15T01:00:00,001880643303671350162
2,001109000286972AC82E9782F34C3A,2421-05-15T01:00:00,001880100768069950130
3,001109000286972AC82E9782F34C3A,2421-05-15T01:01:00,001880643303671350162
4,001109000286972AC82E9782F34C3A,2421-05-15T01:01:00,001880100768069950130


In [36]:
x1['VALUE'] = dataframes['EMAR']['MEDICATION']
x1.head()

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\920256371.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1['VALUE'] = dataframes['EMAR']['MEDICATION']


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,001109000286972AC82E9782F34C3A,2421-05-15T00:00:00,001200501890065450189,Sentil 5mg tab(Clobazam)
1,001109000286972AC82E9782F34C3A,2421-05-15T01:00:00,001880643303671350162,Tazoperan 4.5g inj(Piperacillin/Tazobactam)
2,001109000286972AC82E9782F34C3A,2421-05-15T01:00:00,001880100768069950130,Normal saline 50mL bag 이노엔
3,001109000286972AC82E9782F34C3A,2421-05-15T01:01:00,001880643303671350162,Tazoperan 4.5g inj(Piperacillin/Tazobactam)
4,001109000286972AC82E9782F34C3A,2421-05-15T01:01:00,001880100768069950130,Normal saline 50mL bag 이노엔


In [37]:
my_df_4 = pd.concat([my_df_3, x1], ignore_index=True) 
print(len(my_df_4))
my_df_4.head()

6271621


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08T17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08T19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08T18:15:00,001C_1729_22095,13.0


### Emar Detail Table
### EMAR 에 대한 정보를 담고 있음
### 일단 병합 보류

In [38]:
print(len(dataframes['EMAR_DETAIL']))
dataframes['EMAR_DETAIL'].head()

426543


,SUBJECT_ID,HADM_ID,EMAR_ID,EMAR_SEQ,PARENT_FIELD_ORDINAL,PHARMACY_ID,NOTE_TXT,ON_OFF,INFUSION_RATE_DAY,INFUSION_RATE_HR,INFUSION_RATE_MIN,ITEMID,STAY_ID
0,10900038639,001109000386394405A09C6048648A,001A0000000000000000085251031,1,1,0010000000000000000421613071,NaN,NaN,NaN,NaN,NaN,001200538760021653876,NaN
1,10900038639,001109000386394405A09C6048648A,001A0000000000000000085251114,2,1,0010000000000000000421613070,NaN,NaN,NaN,NaN,NaN,001200538030113453803,NaN
2,10900038639,001109000386394405A09C6048648A,001A0000000000000000085251015,3,1,0010000000000000000421613066,NaN,NaN,NaN,NaN,NaN,001200538340003253834,NaN
3,10900038639,001109000386394405A09C6048648A,001A0000000000000000085262973,4,2,0010000000000000000421661030,NaN,NaN,NaN,NaN,NaN,001880645100773350093,164322.0
4,10900038639,001109000386394405A09C6048648A,001A0000000000000000085263815,5,3,0010000000000000000421661040,NaN,NaN,NaN,NaN,NaN,001200501060040850106,164322.0


In [39]:
# x1 = dataframes['EMAR_DETAIL'].columns
# x2 = my_df_6.columns

# x3 = set(x1) - set(x1 & x2) ## x1.intersection(x2). x1, x2, 공통 col 들
# x3 = list(x3) + ['EMAR_ID']
# print(x3)

In [40]:
# x1 = dataframes['EMAR_DETAIL'][x3]
# my_df_7 = pd.merge(my_df_6, x1, on = 'EMAR_ID', how = 'left')
# print(len(my_df_7))
# my_df_7.head()

### ICU Stays table
### ICU 에 머문 환자들의 입장, 퇴장 시간을 담고 있음
### 각 환자에 대해 여러번 ICU 에 속할 수 있음. 우선 병합하지 않고 보류

In [41]:
print(len(dataframes['ICUSTAYS']))
dataframes['ICUSTAYS'].head()

693


,SUBJECT_ID,HADM_ID,STAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,OP_FLAG
0,10900031317,00110900031317267A8C84B3E742A9,165710.0,EICU,115,2575-05-22T20:06:00,2575-05-29T17:38:00,6.90,0
1,10900015661,0011090001566186F03EA1584D6B6E,165817.0,CCU,NaN,2026-10-07T13:01:00,2026-10-07T14:05:00,0.04,0
2,10900017698,00110900017698691BCD9944D3636F,165496.0,RICU,074,2744-04-04T14:20:00,2744-04-15T21:00:00,11.28,0
3,10900026180,00110900026180490AB95D2DE67366,166269.0,SICU2,052,2138-09-20T17:48:00,2138-09-21T15:20:00,0.90,0
4,10900010079,00110900010079E998FDF7FBB3E952,165531.0,EICU,NaN,2638-05-04T08:55:00,2638-05-26T16:05:00,22.30,0


In [42]:
# x1 = dataframes['ICUSTAYS'] 
# x1[x1['HADM_ID'].duplicated()]

In [43]:
# x1 = dataframes['ICUSTAYS'][['HADM_ID', 'LOS']]
# my_df_8 = pd.merge(my_df_7, x1, on = 'HADM_ID', how = 'left')
# my_df_8

## Hospital table
### 별 정보 담고 있지 않으므로 병합 x

In [44]:
len(dataframes['HOSPITAL'])

1

## Input Events table
### 지속적 주입,  간헐적 수행에 대한 정보 담고있는 table
### 중복 Item ID 없으나 시간정보 있으니 불필요한 col 제외 병합
### 확인 결과 모든 데이터가 start tiem == end time 이므로 start time을 chart time으로 변경해 병합
### amount col 을 value 로 변경

In [45]:
print(len(dataframes['INPUTEVENTS']))
dataframes['INPUTEVENTS'].head()

107747


,INPUTEVENT_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,PATIENTWEIGHT,STORETIME,CGID,ORDERID,LINKORDERID
0,11001223637819,10900073700,00110900073700761AB0F37AC8D0B5,NaN,2037-07-19T00:00:00,2037-07-19T00:00:00,001I_1315_26175,100,cc,NaN,NaN,59.8,2037-07-19T03:36:42,0,0,0
1,11001223650141,10900073700,00110900073700761AB0F37AC8D0B5,NaN,2037-07-19T05:00:00,2037-07-19T05:00:00,001I_1315_26175,300,cc,NaN,NaN,59.8,2037-07-19T05:15:45,0,0,0
2,11001223861493,10900073700,00110900073700761AB0F37AC8D0B5,NaN,2037-07-19T10:00:00,2037-07-19T10:00:00,001I_1315_26175,100,cc,NaN,NaN,59.8,2037-07-19T10:33:59,0,0,0
3,11001223986901,10900073700,00110900073700761AB0F37AC8D0B5,NaN,2037-07-19T12:00:00,2037-07-19T12:00:00,001I_1315_26175,100,cc,NaN,NaN,59.8,2037-07-19T12:36:37,0,0,0
4,11001224004493,10900073700,00110900073700761AB0F37AC8D0B5,NaN,2037-07-19T13:00:00,2037-07-19T13:00:00,001I_1315_26175,650,cc,NaN,NaN,59.8,2037-07-19T12:55:50,0,0,0


In [46]:
x1 = dataframes['INPUTEVENTS']['ITEMID']
my_df_4[my_df_4['ITEMID'].isin(x1)]  ## 이후에 여기서부터 다시 작업하기

,HADM_ID,CHARTTIME,ITEMID,VALUE


In [47]:
cols = my_df_4.columns & dataframes['INPUTEVENTS']
x1 = dataframes['INPUTEVENTS']
x1['is_equal'] = x1['STARTTIME'] == x1['ENDTIME']
print(x1['is_equal'].sum())  ## 모든 store time, end time 값이 같음

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\560473284.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = my_df_4.columns & dataframes['INPUTEVENTS']


107747


In [48]:
x1['CHARTTIME'] = x1['STORETIME']
x1['VALUE'] = x1['AMOUNT']

cols = x1.columns & my_df_4.columns
x1 = x1[cols]
x1.head()

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\3902290661.py:4: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = x1.columns & my_df_4.columns


,HADM_ID,ITEMID,CHARTTIME,VALUE
0,00110900073700761AB0F37AC8D0B5,001I_1315_26175,2037-07-19T03:36:42,100
1,00110900073700761AB0F37AC8D0B5,001I_1315_26175,2037-07-19T05:15:45,300
2,00110900073700761AB0F37AC8D0B5,001I_1315_26175,2037-07-19T10:33:59,100
3,00110900073700761AB0F37AC8D0B5,001I_1315_26175,2037-07-19T12:36:37,100
4,00110900073700761AB0F37AC8D0B5,001I_1315_26175,2037-07-19T12:55:50,650


In [49]:
# empty_list = []
# remove_list = ['INPUTEVENT_ID', 'ENDTIME', 'STORETIME']
# for col in dataframes['INPUTEVENTS'].columns:
#     print(col)
#     print('column :{} 의 unique 개수 : {}'.format(col,len(dataframes['INPUTEVENTS'][col].unique())))
#     if len(dataframes['INPUTEVENTS'][col].unique()) == 1:
#         empty_list.append(col)
#     if col in remove_list:
#         empty_list.append(col)

In [50]:
# x1 = dataframes['INPUTEVENTS'].drop(empty_list, axis = 1)
# x1.rename(columns = {'STARTTIME':'CHARTTIME', 'PATIENTWEIGHT':'WEIGHT'}, inplace = True) # inplace 인자 안주면 변경x  
# x1.head()

In [51]:
my_df_5 = pd.concat([my_df_4, x1], ignore_index = True)
print(len(my_df_5))
my_df_5.head()

6379368


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08T17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08T19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08T18:15:00,001C_1729_22095,13.0


## Med Recon table
### 약 정보를 담고 있는 테이블
### Hadm ID 가 없어 어떤 환자에게 투여했는지 모르므로 병합 x

In [52]:
print(len(dataframes['MEDRECON']['STAY_ID']))
dataframes['MEDRECON'].head()

154


,STAY_ID,CHARTTIME,NAME,GSN,NDC
0,00110900015189EF6388928B594AA8,2716-05-03T18:41:34,Gaster 20mg tab(Famotidine) 1 tab [P....,NaN,NaN
1,001109000175268A82EFDE5509DDAA,2513-07-15T13:23:26,[IMC]\r\nCardiazem 30mg tab(Diltiazem HCl) ...,NaN,NaN
2,00110900008222F4E68F69EEA3E8CC,2198-09-30T18:14:58,본원 NR\r\nETRAVIL 10MG TAB(AMITRIPTYLINE) ...,NaN,NaN
3,0011090003786369EB073EAE472376,2529-07-13T01:10:51,IMC>\r\nASPIRIN PROTECT 100MG TAB(ASPIRIN ENTE...,NaN,NaN
4,0011090003029329528E1055FB869E,2790-04-21T00:32:21,*** 처방 약\r\nASCITE GRAN(L-ISOLEUCINE) 4.1G 1P ...,NaN,NaN


In [103]:
print(dataframes['MEDRECON']['GSN'].nunique())
print(dataframes['MEDRECON']['NDC'].nunique())

0
0


In [53]:
# x1 = dataframes['MEDRECON']['STAY_ID']
# my_df_8[my_df_8['STAY_ID'].isin(x1)]

## MicroBiologyEvents
### 어떤 항생제가 효과적인지 확인하는 테이블 
### Item ID 가 존재하지만, docu 확인했을 때 기존의 테이블의 Item ID  = AB Item ID 인 것 같음
### AB item id 를  item id 로 변경, quantity 를 value 로 바꿔 병합

In [99]:
print(len(dataframes['MICROBIOLOGYEVENTS']))
dataframes['MICROBIOLOGYEVENTS'].head()

17034


,MICROEVENT_ID,SUBJECT_ID,HADM_ID,MICRO_SPECIMEN_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,TEST_SEQ,STOREDATE,...,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION,COMMENTS,ORDERDATE,ORDER_COMMENTS,STAY_ID,ITEMID,VALUE
0,00122090119066L401620220902100601N111,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,NaN,NaN,NaN,-,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,ESBL,3X10³/mL
1,00122090119066L401620220902100601N112,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,32>=,>=,32,R,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,AMP,3X10³/mL
2,00122090119066L401620220902100601N113,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,4,NaN,4,S,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,AMC,3X10³/mL
3,00122090119066L401620220902100601N114,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,16,NaN,16,S,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,TZP,3X10³/mL
4,00122090119066L401620220902100601N115,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,4<=,<=,4,S,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,CZ06,3X10³/mL


In [55]:
x1 = dataframes['MICROBIOLOGYEVENTS']
cols = x1.columns & my_df_5.columns
cols

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\1168354646.py:2: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = x1.columns & my_df_5.columns


Index(['HADM_ID', 'CHARTTIME'], dtype='object')

In [56]:
x1['ITEMID'] = x1['AB_ITEMID']
x1['VALUE'] = x1['QUANTITY']

In [57]:
x1.head()

,MICROEVENT_ID,SUBJECT_ID,HADM_ID,MICRO_SPECIMEN_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,TEST_SEQ,STOREDATE,...,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION,COMMENTS,ORDERDATE,ORDER_COMMENTS,STAY_ID,ITEMID,VALUE
0,00122090119066L401620220902100601N111,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,NaN,NaN,NaN,-,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,ESBL,3X10³/mL
1,00122090119066L401620220902100601N112,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,32>=,>=,32,R,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,AMP,3X10³/mL
2,00122090119066L401620220902100601N113,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,4,NaN,4,S,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,AMC,3X10³/mL
3,00122090119066L401620220902100601N114,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,16,NaN,16,S,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,TZP,3X10³/mL
4,00122090119066L401620220902100601N115,10900087730,00110900087730929067646F0B6970,22090119066,2309-07-11T00:00:00,2309-07-11T17:20:16,2URV,"Urine, Voided",1,2309-07-16T00:00:00,...,4<=,<=,4,S,Nitrofurantoin disc: 11mm( R ),2309-07-11T00:00:00,NaN,NaN,CZ06,3X10³/mL


In [58]:
x1 = x1[cols]

my_df_6 = pd.concat([my_df_5, x1], ignore_index = True)
print(len(my_df_6))
my_df_6.head()

6396402


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08T17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08T19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08T18:15:00,001C_1729_22095,13.0


In [59]:
# empty_list = []
# remove_list = ['MICROEVENT_ID']
# for col in dataframes['MICROBIOLOGYEVENTS'].columns:
#     print(col)
#     print('column :{} 의 unique 개수 : {}'.format(col,len(dataframes['MICROBIOLOGYEVENTS'][col].unique())))
#     if len(dataframes['MICROBIOLOGYEVENTS'][col].unique()) == 1:
#         empty_list.append(col)
#     if col in remove_list:
#         empty_list.append(col)

In [60]:
# my_df_9 = pd.concat([my_df_8, x1], ignore_index = True)
# print(len(my_df_9))
# my_df_9.head()

## Output Events table
### Item ID, 시간 정보 병합하기

In [61]:
print(len(dataframes['OUTPUTEVENTS']))
dataframes['OUTPUTEVENTS'].head()

126217


,OUTPUTEVENT_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,STORETIME,ITEMID,VALUE,VALUEUOM,WARNING
0,11001287695255,10900062288,001109000622887948C8727CE7AA0C,166612.0,2336-08-17T23:47:00,2336-08-18T00:29:02,001O_1481_25470,0.0,cc,0
1,11001289483557,10900062288,001109000622887948C8727CE7AA0C,NaN,2336-08-20T23:00:00,2336-08-20T23:47:49,001O_1479_25880,2006.0,cc,0
2,11001290285825,10900062288,001109000622887948C8727CE7AA0C,NaN,2336-08-21T23:00:00,2336-08-21T23:59:53,001O_1479_25880,2050.0,cc,0
3,11001287703706,10900062288,001109000622887948C8727CE7AA0C,166612.0,2336-08-18T00:00:00,2336-08-18T01:23:38,001O_1481_25470,0.0,cc,0
4,11001287709086,10900062288,001109000622887948C8727CE7AA0C,166612.0,2336-08-18T01:00:00,2336-08-18T02:15:54,001O_1481_25470,0.0,cc,0


In [62]:
x1 = dataframes['OUTPUTEVENTS']['ITEMID']
common_subjects = my_df_6[my_df_6['ITEMID'].isin(x1)]
common_subjects

,HADM_ID,CHARTTIME,ITEMID,VALUE


In [63]:
x1 = dataframes['OUTPUTEVENTS']
cols = x1.columns & my_df_6.columns

x1 = x1[cols]
my_df_7 = pd.concat([my_df_6, x1], ignore_index = True)
print(len(my_df_7))
my_df_7.head()

6522619


C:\Users\choij\AppData\Local\Temp\ipykernel_4712\715420289.py:2: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = x1.columns & my_df_6.columns


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08T17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08T19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08T18:15:00,001C_1729_22095,13.0


## Patients table
### 환자의 정보를 담고 있는 Table
### 측정값이 일단 없으므로 병합 X


In [64]:
print(len(dataframes['PATIENTS']))
dataframes['PATIENTS'].head()

590


,SUBJECT_ID,SEX,ANCHOR_AGE,ANCHOR_YEAR,ANCHOR_YEAR_GROUP,DOD
0,10900028075,F,73 years,2355,2022-2024,NaN
1,10900023253,M,61 years,2458,2021-2023,NaN
2,10900007563,M,64 years,2432,2022-2024,NaN
3,10900011021,M,54 years,2613,2021-2023,NaN
4,10900039519,M,83 years,2091,2021-2023,NaN


In [65]:
x1 = dataframes['PATIENTS']
x1.drop_duplicates(subset= ['SUBJECT_ID'], keep = 'first', inplace = True)
x1

,SUBJECT_ID,SEX,ANCHOR_AGE,ANCHOR_YEAR,ANCHOR_YEAR_GROUP,DOD
0,10900028075,F,73 years,2355,2022-2024,NaN
1,10900023253,M,61 years,2458,2021-2023,NaN
2,10900007563,M,64 years,2432,2022-2024,NaN
3,10900011021,M,54 years,2613,2021-2023,NaN
4,10900039519,M,83 years,2091,2021-2023,NaN
...,...,...,...,...,...,...
585,10900048167,M,62 years,2248,2020-2022,NaN
586,10900088018,M,48 years,2237,2021-2023,2237-09-11T00:00:00
587,10900091004,M,42 years,2758,2022-2024,2759-05-08T00:00:00
588,10900092879,F,1 days,2919,2022-2024,NaN


In [66]:
# my_df_11 = pd.merge(my_df_10, x1, on = 'SUBJECT_ID')
# my_df_11

## Services table
### 별 다른 정보가 없다고 판단해 병합 x

In [67]:
x1 = dataframes['SERVICES']
print(len(x1))
x1.head()

1059


,SUBJECT_ID,HADM_ID,TRANSFERTIME,PREV_SERVICE,CURR_SERVICE
0,10900010641,001109000106411DAE1226BA38411E,2360-07-19T00:00:00,NaN,IMC
1,10900002327,00110900002327D20623225609DBF9,2499-05-25T00:00:00,NaN,IMC
2,10900002327,00110900002327D20623225609DBF9,2499-06-07T00:00:00,IMC,IMN
3,10900003302,00110900003302379C5CB71E10508A,2714-05-02T00:00:00,NaN,IMC
4,10900044531,001109000445317E34FD11C3769A2D,2805-04-08T00:00:00,NaN,NS


## Prescriptions table
### 처방한 약에 대한 정보 담고 있는 테이블
### 행으로 병합
### 해당 테이블에는 많은 Item ID 가 기존 테이블과 겹치므로 겹치지 않는 item id 를 갖는 것만 저장

In [93]:
x1 = dataframes['PRESCRIPTIONS']
print(len(x1))
x1.head()

202078


,SUBJECT_ID,HADM_ID,PHARMACY_ID,STARTTIME,STOPTIME,DRUG_TYPE,DRUG,GSN,NDC,PROD_STRENGTH,FORM_RX,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,DOSES_PER_24_HRS,ROUTE,ITEMID,STAY_ID
0,10900087730,00110900087730929067646F0B6970,0010000000000000000435520415,2309-08-12T00:00:00,2309-08-12T00:00:00,NaN,Dextrose 5% 50mL bag 이노엔,B05BA03,640001361,x1,bag,50.0,ml,1.0,bag,NaN,IV,1880100768068250130,NaN
1,10900087730,00110900087730929067646F0B6970,0010000000000000000435094602,2309-08-09T00:00:00,2309-08-09T00:00:00,BASE,Dextrose 5% 200mL bag 이노엔,B05BA03,640001381,q24h,bag,200.0,ml,200.0,ml,NaN,IV,1880100768054550130,NaN
2,10900011804,0011090001180456B85EBD007F3149,0010000000000000000434180101,2879-03-20T00:00:00,2879-03-20T00:00:00,NaN,Myungdopar 125mg tab(L-Dopa/benserazide),N04BA02,651905030,tid ++,tab,125.0,mg,1.0,tab,NaN,Oral,1200501070096250107,166045.0
3,10900074415,00110900074415020B0797B2C6A9A0,0010000000000000000434279708,2787-04-12T00:00:00,2787-04-12T00:00:00,NaN,Plasma solution A 1000mL bag,B05BB04,640002610,x1,bag,1.0,l,1.0,l,NaN,IV,1880100768084250130,NaN
4,10900088507,001109000885074EEE74D3E01DCFCC,0010000000000000000434324790,2995-02-21T00:00:00,2995-02-21T00:00:00,MAIN,Dextrose 5% & Na K2 1L bag(D5WNa77K20) 중외,B05BB02,644900061,x1,bag,1.0,l,1.0,l,NaN,IV,1880644900061650132,NaN


In [94]:
x1['DRUG'].nunique()

1264

In [69]:
print(len(my_df_7[my_df_7['ITEMID'].isin(x1['ITEMID'])])) ## 약 42만개의 데이터에서 item id 겹침
print(my_df_7[my_df_7['ITEMID'].isin(x1['ITEMID'])].nunique())

426282
HADM_ID         590
CHARTTIME    160199
ITEMID         1243
VALUE          1245
dtype: int64


In [70]:
x1_filtered = x1[~x1['ITEMID'].isin(my_df_7['ITEMID'])]
len(x1_filtered)

15730

In [71]:
x1_filtered['VALUE'] = x1_filtered['DOSE_VAL_RX']
x1_filtered['CHARTTIME'] = x1_filtered['STARTTIME']

cols = x1_filtered.columns & my_df_7.columns
x1_filtered = x1_filtered[cols]

my_df_8 = pd.concat([my_df_7, x1_filtered], ignore_index = True)
print(len(my_df_8))

6538349


C:\Users\choij\AppData\Local\Temp\ipykernel_4712\1329492903.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_filtered['VALUE'] = x1_filtered['DOSE_VAL_RX']
C:\Users\choij\AppData\Local\Temp\ipykernel_4712\1329492903.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_filtered['CHARTTIME'] = x1_filtered['STARTTIME']
C:\Users\choij\AppData\Local\Temp\ipykernel_4712\1329492903.py:4: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation 

## ProcedureceEvents table
### MIMIC 에는 나와있지 않지만 아마 어떠한 투여 관련 나타내는 테이블
### 불필요한 col 날린 후 행으로 병합

In [72]:
x1 = dataframes['PROCEDUREEVENTS']
print(len(x1))
x1.head()

2025


,PROCEDUREEVENT_ID,SUBJECT_ID,HADM_ID,STAY_ID,STARTTIME,ENDTIME,STORETIME,ITEMID,VALUE,VALUEUOM,LOCATION,LOCATIONCATEGORY,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,PATIENTWEIGHT
0,001441559993,10900078991,00110900078991AABBDBDCA37982CC,167004.0,2640-06-17T00:50:00,2640-06-17T03:35:00,2640-06-17T12:40:36,001P_H3603,NaN,NaN,NaN,NaN,수술,NaN,NaN,0.0
1,001100429587_O,10900078991,00110900078991AABBDBDCA37982CC,167004.0,2640-06-16T23:54:42,2640-06-17T03:45:00,NaN,001P_OR,NaN,NaN,NaN,NaN,수술,NaN,NaN,64.0
2,001441007522,10900072482,001109000724828F174D66710CB42F,166929.0,2865-04-21T09:00:00,2865-04-21T13:23:48,2865-04-21T13:44:08,001P_H34412,NaN,NaN,NaN,NaN,수술,NaN,NaN,65.1
3,001100426854_A,10900059549,0011090005954957C0791EB4994ECA,166640.0,2281-09-01T13:05:00,2281-09-01T19:00:00,NaN,001P_Anesthesia,NaN,NaN,NaN,NaN,수술,NaN,NaN,57.4
4,001100427808_A,10900051157,00110900051157559BEA3087B89DE7,166915.0,2275-09-11T14:40:00,2275-09-11T22:05:00,NaN,001P_Anesthesia,NaN,NaN,NaN,NaN,수술,NaN,NaN,53.8


In [73]:
my_df_8[my_df_8['ITEMID'].isin(x1)]

,HADM_ID,CHARTTIME,ITEMID,VALUE


In [74]:
cols = my_df_8.columns & x1.columns
cols

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\2696570973.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = my_df_8.columns & x1.columns


Index(['HADM_ID', 'ITEMID', 'VALUE'], dtype='object')

In [75]:
cols = my_df_8.columns & x1.columns

x1 = x1[cols]
x1['CHARTTIME'] = dataframes['PROCEDUREEVENTS']['STORETIME']
my_df_9 = pd.concat([my_df_8, x1], ignore_index = True)
len(my_df_9)

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\1300825500.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = my_df_8.columns & x1.columns
C:\Users\choij\AppData\Local\Temp\ipykernel_4712\1300825500.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1['CHARTTIME'] = dataframes['PROCEDUREEVENTS']['STORETIME']


6540374

In [76]:
my_df_9

,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08T17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08T19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08T18:15:00,001C_1729_22095,13.0
...,...,...,...,...
6540369,001109000885074EEE74D3E01DCFCC,NaN,001P_Anesthesia,NaN
6540370,00110900074415020B0797B2C6A9A0,NaN,001P_OR,NaN
6540371,001109000204261546203818BC4A1E,NaN,001P_Surgery,NaN
6540372,00110900086019BDD664499C01200B,2117-09-21T21:00:46,001P_H34422,NaN


## Procedures ICD table
### 환자가 청구받을 시술들 정보 담은 테이블
### 가격 정보를 담고 있는 듯 하니 일단 병합 x

In [77]:
x1 = dataframes['PROCEDURES_ICD']
x1

,SUBJECT_ID,HADM_ID,SEQ_NUM,CHARTDATE,ICD_CODE,ICD_VERSION,ITEMID,STAY_ID
0,10900044531,001109000445317E34FD11C3769A2D,1,2805-04-09T00:00:00,39.72,ICD9CM,001P00001315,166226.0
1,10900091813,001109000918134430DA2FEF66F1BD,1,2316-07-31T00:00:00,39.28,ICD9CM,001P00001331,166032.0
2,10900058227,00110900058227CFA89794BD56C2E6,1,2866-03-23T00:00:00,76.69,ICD9CM,001P00004390,166151.0
3,10900089564,00110900089564BA3F157434A45960,1,2883-12-18T00:00:00,54.11,ICD9CM,001P00002818,NaN
4,10900089564,00110900089564BA3F157434A45960,2,2883-12-18T00:00:00,54.11,ICD9CM,001P00002818,NaN
...,...,...,...,...,...,...,...,...
408,10900056372,00110900056372B60A6184169BC440,1,2669-05-13T00:00:00,5.90,ICD9CM,001P00002171,NaN
409,10900056372,00110900056372B60A6184169BC440,2,2669-06-07T00:00:00,31.48,ICD9CM,001P00007225,NaN
410,10900093831,0011090009383177C7AA383A16406C,1,3011-03-18T00:00:00,6.70,ICD9CM,001P00002622,166946.0
411,10900065757,00110900065757432673D056532F5D,1,2930-04-04T00:00:00,32.40,ICD9CM,001P00000614,166825.0


In [78]:
my_df_9[my_df_9['ITEMID'].isin(x1)]

,HADM_ID,CHARTTIME,ITEMID,VALUE


In [79]:
x1['SUBJECT_ID'].nunique()

294

## Test Events table
### MIMIC docu 에 따로 x
### value col 이 없는 것으로 보아 따로 병합 x

In [80]:
x1 = dataframes['TESTEVENTS']
x1

,TESTEVENT_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,STORETIME,FILE_PATH,STAY_ID
0,1202209300041299139,10900020426,001109000204261546203818BC4A1E,001RG2012,2628-05-22T23:59:45,2628-05-23T09:46:00,109000204262628052223590347,166209.0
1,1202209300041299136,10900020426,001109000204261546203818BC4A1E,001RG2012,2628-05-23T05:20:09,2628-05-23T09:46:00,109000204262628052305200305,166209.0
2,1202210030041313819,10900020426,001109000204261546203818BC4A1E,001RG2012,2628-05-23T21:47:07,2628-05-26T15:55:00,109000204262628052321470311,NaN
3,1202210030041313820,10900020426,001109000204261546203818BC4A1E,001RG2012,2628-05-25T10:54:45,2628-05-26T15:55:00,109000204262628052510540301,NaN
4,1202209300041301967,10900020426,001109000204261546203818BC4A1E,001RC102,2628-05-23T05:20:15,2628-05-23T11:07:00,109000204262628052305200307,166209.0
...,...,...,...,...,...,...,...,...
14578,1202210240041516802,10900063324,0011090006332408D999877D2BE1C1,001RG2012,2638-06-12T23:40:18,2638-06-14T18:43:00,109000633242638061223400325,NaN
14579,1202210240041506011,10900063324,0011090006332408D999877D2BE1C1,001RG3011,2638-06-12T10:30:38,2638-06-14T09:41:00,109000633242638061210300307,NaN
14580,1202210240041506012,10900063324,0011090006332408D999877D2BE1C1,001RG3011,2638-06-13T12:41:21,2638-06-14T09:41:00,109000633242638061312410328,NaN
14581,1202210240041506010,10900063324,0011090006332408D999877D2BE1C1,001RG3011,2638-06-13T20:38:38,2638-06-14T09:41:00,109000633242638061320380385,166800.0


## Transfers
### 환자들이 이동한 병실에 대한 정보를 담고있는 테이블
### 병합 x

In [81]:
x1 = dataframes['TRANSFERS']
x1

,SUBJECT_ID,HADM_ID,TRANSFER_ID,EVENTTYPE,CAREUNIT,INTIME,OUTTIME
0,10900019700,001109000197006ADD0C2633B5765F,001109000197006ADD0C2633B5765F01,ed,ER99,2664-03-27T18:23:00,2664-03-28T01:34:00
1,10900019700,001109000197006ADD0C2633B5765F,001109000197006ADD0C2633B5765F02,admit,051,2664-03-28T01:35:00,2664-03-28T20:09:00
2,10900019700,001109000197006ADD0C2633B5765F,001109000197006ADD0C2633B5765F03,transfer,SICU2,2664-03-28T20:10:00,2664-03-30T18:40:00
3,10900019700,001109000197006ADD0C2633B5765F,001109000197006ADD0C2633B5765F04,transfer,051,2664-03-30T18:41:00,2664-04-12T16:42:00
4,10900019700,001109000197006ADD0C2633B5765F,001109000197006ADD0C2633B5765F05,transfer,SICU2,2664-04-12T16:43:00,2664-04-20T15:05:00
...,...,...,...,...,...,...,...
2373,10900071957,00110900071957F7E8019AEA85BAB9,00110900071957F7E8019AEA85BAB901,discharge,NICU,2211-09-09T00:00:00,NaN
2374,10900088972,001109000889724B5F141EC173A2A5,001109000889724B5F141EC173A2A500,admit,NICU,2144-09-23T21:48:00,2144-09-24T12:37:00
2375,10900088972,001109000889724B5F141EC173A2A5,001109000889724B5F141EC173A2A501,discharge,NICU,2144-09-24T12:38:00,NaN
2376,10900092974,00110900092974F0121B89316F94D4,00110900092974F0121B89316F94D400,admit,NICU,2182-09-14T21:58:00,2182-09-15T12:37:00


## Triage
### ED 에 있는 환자들에 대한 정보
### 일단 병합 x

In [82]:
x1 = dataframes['TRIAGE']
x1

,STAY_ID,TEMP,HR,RR,SAO2,PAIN,ACUITY,SBP,DBP
0,001109000613356DCE7C1859B5679D,37.1,118,23,90,무,3,146,83
1,001109000147182FA97BAE4C81C2AC,37.0,105,20,97,평가불가,1,124,60
2,001109000927416563B4759AF47309,36.8,156,40,98,평가불가,2,-1,-1
3,00110900077026850A9F6A5F707EF3,39.3,104,18,98,무,3,98,56
4,0011090004525405B16CCBEA059CA7,37.3,-1,-1,73,무,1,-1,-1
...,...,...,...,...,...,...,...,...,...
179,00110900002220327E4822D6697B13,36.6,73,16,99,유,3,145,82
180,00110900005807599F01D749BC97C2,36.7,97,18,96,유,2,160,65
181,001109000701593A89C3C41A199398,38.1,138,32,99,무,1,108,51
182,00110900048149271298B1887704B4,36.6,101,36,80,무,1,163,98


### 모든 테이블 병합
#### 총 650만개의 데이터, 4개의 col

In [83]:
print(len(my_df_9))
my_df_9.head()

6540374


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08T17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08T19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08T18:15:00,001C_1729_22095,13.0


In [84]:
import time

In [85]:
custom_my_df = my_df_9[:1000000]

In [86]:
print(custom_my_df.duplicated(subset=['HADM_ID', 'CHARTTIME', 'ITEMID']).sum()) ## 총 12600 개의 동일 item id 존재
custom_my_df = custom_my_df.drop_duplicates(subset=['HADM_ID', 'CHARTTIME', 'ITEMID'])
custom_my_df.duplicated(subset=['HADM_ID', 'CHARTTIME', 'ITEMID']).sum()

12600


0

In [87]:
start = time.time()
group_data = custom_my_df.groupby(['HADM_ID', 'CHARTTIME']).apply(lambda x: x.set_index('ITEMID')[['VALUE']]).unstack()
end = (time.time() - start) / 60

In [88]:
print(end) #100 만개의 데이터 다루는데 16분

15.929240731398265


In [89]:
group_data  # 총 4066 개의 feature 존재 . artifact removal 통해 제거 필요 
## 5분으로 resample 과정에서 값들을 sum 하기 위해서는 int 필요

VALUE  \
ITEMID                                             001C_1002_27090   
HADM_ID                        CHARTTIME                             
00110900002214B06A868896938A2A 2904-04-07T23:38:00             NaN   
                               2904-04-08T03:21:00             NaN   
                               2904-04-08T04:00:00             NaN   
                               2904-04-08T07:23:00             NaN   
                               2904-04-08T07:27:00             NaN   
...                                                            ...   
00110900027557126B9DA290D47DA1 2711-05-27T21:10:00             NaN   
                               2711-05-27T21:39:00             NaN   
                               2711-05-27T23:11:00             NaN   
                               2711-05-28T05:00:00             NaN   
                               2711-05-28T05:06:00             NaN   

                                                                    \
ITEMID                                             001C_1003_24480   
HADM_ID                        CHARTTIME                             
00110900002214B06A868896938A2A 2904-04-07T23:38:00             100   
                               2904-04-08T03:21:00             NaN   
                               2904-04-08T04:00:00              97   
                               2904-04-08T07:23:00              98   
                               2904-04-08T07:27:00              97   
...                                                            ...   
00110900027557126B9DA290D47DA1 2711-05-27T21:10:00             NaN   
                               2711-05-27T21:39:00             NaN   
                               2711-05-27T23:11:00             NaN   
                               2711-05-28T05:00:00             NaN   
                               2711-05-28T05:06:00             NaN   

                                                                    \
ITEMID                                             001C_1006_27225   
HADM_ID                        CHARTTIME                             
00110900002214B06A868896938A2A 2904-04-07T23:38:00             NaN   
                               2904-04-08T03:21:00             NaN   
                               2904-04-08T04:00:00             NaN   
                               2904-04-08T07:23:00             196   
                               2904-04-08T07:27:00             NaN   
...                                                            ...   
00110900027557126B9DA290D47DA1 2711-05-27T21:10:00             NaN   
                               2711-05-27T21:39:00             NaN   
                               2711-05-27T23:11:00             NaN   
                               2711-05-28T05:00:00             NaN   
                               2711-05-28T05:06:00             NaN   

                                                                    \
ITEMID                                             001C_1012_24795   
HADM_ID                        CHARTTIME                             
00110900002214B06A868896938A2A 2904-04-07T23:38:00             185   
                               2904-04-08T03:21:00             156   
                               2904-04-08T04:00:00             138   
                               2904-04-08T07:23:00              91   
                               2904-04-08T07:27:00             111   
...                                                            ...   
00110900027557126B9DA290D47DA1 2711-05-27T21:10:00             NaN   
                               2711-05-27T21:39:00             NaN   
                               2711-05-27T23:11:00             NaN   
                               2711-05-28T05:00:00             NaN   
                               2711-05-28T05:06:00             NaN   

                                                                    \
ITEMID                                             001C_1013_25640 

In [90]:
## 시간 데이터 전처리

In [302]:
x1 = custom_my_df[:5000] # 날짜에서 연도 데이터는 중요X
x1

,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08T21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08T19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08T17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08T19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08T18:15:00,001C_1729_22095,13.0
...,...,...,...,...
5064,00110900002327D20623225609DBF9,2499-06-02T08:13:00,001D_10771_10065,Internal Jugular Catheter 삽입부위 oozing 없음
5065,00110900002327D20623225609DBF9,2499-05-30T13:22:00,001D_6499_12710,처방에 의해 약물을 투여함
5066,00110900002327D20623225609DBF9,2499-06-13T08:57:00,001D_1784_12690,통증 있음
5067,00110900002327D20623225609DBF9,2499-06-02T00:22:00,001D_4964_10615,낙상방지 교육함


In [303]:
x1['CHARTTIME'] = x1['CHARTTIME'].str.replace('T', ' ')
x1

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\2697696275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1['CHARTTIME'] = x1['CHARTTIME'].str.replace('T', ' ')


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,2904-04-08 21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,2904-04-08 19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,2904-04-08 17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,2904-04-08 19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,2904-04-08 18:15:00,001C_1729_22095,13.0
...,...,...,...,...
5064,00110900002327D20623225609DBF9,2499-06-02 08:13:00,001D_10771_10065,Internal Jugular Catheter 삽입부위 oozing 없음
5065,00110900002327D20623225609DBF9,2499-05-30 13:22:00,001D_6499_12710,처방에 의해 약물을 투여함
5066,00110900002327D20623225609DBF9,2499-06-13 08:57:00,001D_1784_12690,통증 있음
5067,00110900002327D20623225609DBF9,2499-06-02 00:22:00,001D_4964_10615,낙상방지 교육함


In [304]:
x1['CHARTTIME'] = x1['CHARTTIME'].apply(lambda x: x[5:])
x1

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\945476401.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1['CHARTTIME'] = x1['CHARTTIME'].apply(lambda x: x[5:])


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,04-08 21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,04-08 19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,04-08 17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,04-08 19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,04-08 18:15:00,001C_1729_22095,13.0
...,...,...,...,...
5064,00110900002327D20623225609DBF9,06-02 08:13:00,001D_10771_10065,Internal Jugular Catheter 삽입부위 oozing 없음
5065,00110900002327D20623225609DBF9,05-30 13:22:00,001D_6499_12710,처방에 의해 약물을 투여함
5066,00110900002327D20623225609DBF9,06-13 08:57:00,001D_1784_12690,통증 있음
5067,00110900002327D20623225609DBF9,06-02 00:22:00,001D_4964_10615,낙상방지 교육함


In [305]:
x1['CHARTTIME'] = pd.to_datetime(x1['CHARTTIME'], format = '%m-%d %H:%M:%S')
x1

C:\Users\choij\AppData\Local\Temp\ipykernel_4712\4118685935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1['CHARTTIME'] = pd.to_datetime(x1['CHARTTIME'], format = '%m-%d %H:%M:%S')


,HADM_ID,CHARTTIME,ITEMID,VALUE
0,00110900002214B06A868896938A2A,1900-04-08 21:00:00,001C_1728_22200,19.4
1,00110900002214B06A868896938A2A,1900-04-08 19:20:00,001C_1729_22095,12.8
2,00110900002214B06A868896938A2A,1900-04-08 17:10:00,001C_1729_22095,9.2
3,00110900002214B06A868896938A2A,1900-04-08 19:00:00,001C_1729_22095,12.7
4,00110900002214B06A868896938A2A,1900-04-08 18:15:00,001C_1729_22095,13.0
...,...,...,...,...
5064,00110900002327D20623225609DBF9,1900-06-02 08:13:00,001D_10771_10065,Internal Jugular Catheter 삽입부위 oozing 없음
5065,00110900002327D20623225609DBF9,1900-05-30 13:22:00,001D_6499_12710,처방에 의해 약물을 투여함
5066,00110900002327D20623225609DBF9,1900-06-13 08:57:00,001D_1784_12690,통증 있음
5067,00110900002327D20623225609DBF9,1900-06-02 00:22:00,001D_4964_10615,낙상방지 교육함


In [342]:
x2 = x1
x2 = x2.groupby(['HADM_ID', 'CHARTTIME']).resample('5T', on = 'CHARTTIME').apply(lambda x: x.set_index('ITEMID')[['VALUE']]).unstack()
x2 = x2.droplevel(1, axis=0) 
x2

VALUE  \
ITEMID                                             001C_1003_24480   
HADM_ID                        CHARTTIME                             
00110900002214B06A868896938A2A 1900-04-07 23:35:00             100   
                               1900-04-08 03:20:00             NaN   
                               1900-04-08 04:00:00              97   
                               1900-04-08 07:20:00              98   
                               1900-04-08 07:25:00              97   
...                                                            ...   
00110900002327D20623225609DBF9 1900-06-23 17:15:00             NaN   
                               1900-06-23 17:40:00             NaN   
                               1900-06-23 21:05:00             NaN   
                               1900-06-23 23:25:00             NaN   
                               1900-06-24 08:55:00             NaN   

                                                                    \
ITEMID                                             001C_1006_27225   
HADM_ID                        CHARTTIME                             
00110900002214B06A868896938A2A 1900-04-07 23:35:00             NaN   
                               1900-04-08 03:20:00             NaN   
                               1900-04-08 04:00:00             NaN   
                               1900-04-08 07:20:00             196   
                               1900-04-08 07:25:00             NaN   
...                                                            ...   
00110900002327D20623225609DBF9 1900-06-23 17:15:00             NaN   
                               1900-06-23 17:40:00             NaN   
                               1900-06-23 21:05:00             NaN   
                               1900-06-23 23:25:00             NaN   
                               1900-06-24 08:55:00             NaN   

                                                                    \
ITEMID                                             001C_1012_24795   
HADM_ID                        CHARTTIME                             
00110900002214B06A868896938A2A 1900-04-07 23:35:00             185   
                               1900-04-08 03:20:00             156   
                               1900-04-08 04:00:00             138   
                               1900-04-08 07:20:00              91   
                               1900-04-08 07:25:00             111   
...                                                            ...   
00110900002327D20623225609DBF9 1900-06-23 17:15:00             NaN   
                               1900-06-23 17:40:00             NaN   
                               1900-06-23 21:05:00             NaN   
                               1900-06-23 23:25:00             NaN   
                               1900-06-24 08:55:00             NaN   

                                                                    \
ITEMID                                             001C_1013_25640   
HADM_ID                        CHARTTIME                             
00110900002214B06A868896938A2A 1900-04-07 23:35:00              80   
                               1900-04-08 03:20:00             133   
                               1900-04-08 04:00:00              74   
                               1900-04-08 07:20:00              63   
                               1900-04-08 07:25:00              85   
...                                                            ...   
00110900002327D20623225609DBF9 1900-06-23 17:15:00             NaN   
                               1900-06-23 17:40:00             NaN   
                               1900-06-23 21:05:00             NaN   
                               1900-06-23 23:25:00             NaN   
                               1900-06-24 08:55:00             NaN   

                                                                    \
ITEMID                                             001C_1014_27095 